In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../artifacts/sentiment_analysis.csv")

In [3]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


##Preprocessing


In [10]:
#data preprocess
data.duplicated().sum()


np.int64(0)

In [11]:
data.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [12]:
import re
import string

In [14]:
data['tweet']

0       #fingerprint #Pregnancy Test https://goo.gl/h1...
1       Finally a transparant silicon case ^^ Thanks t...
2       We love this! Would you go? #talk #makememorie...
3       I'm wired I know I'm George I was made that wa...
4       What amazing service! Apple won't even talk to...
                              ...                        
7915    Live out loud #lol #liveoutloud #selfie #smile...
7916    We would like to wish you an amazing day! Make...
7917    Helping my lovely 90 year old neighbor with he...
7918    Finally got my #smart #pocket #wifi stay conne...
7919    Apple Barcelona!!! #Apple #Store #BCN #Barcelo...
Name: tweet, Length: 7920, dtype: object

In [15]:
data['tweet'] = data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [16]:
data['tweet'].head()

0    #fingerprint #pregnancy test https://goo.gl/h1...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

In [19]:
#remove links
data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))

In [20]:
data['tweet'].head()

0    #fingerprint #pregnancy test  #android #apps #...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

In [29]:
#remove punctuations
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,"")

    return text
data['tweet'] = data['tweet'].apply(remove_punctuations)
data['tweet'].head()

0    fingerprint pregnancy test  android apps beaut...
1    finally a transparant silicon case  thanks to ...
2    we love this  would you go talk makememories u...
3    im wired i know im george i was made that way ...
4    what amazing service  apple wont even talk to ...
Name: tweet, dtype: object

In [34]:
#remove numbers
data['tweet'] = data['tweet'].str.replace("\\d+", "",regex=True)
data['tweet'].tail()
data['tweet'].head()

0    fingerprint pregnancy test  android apps beaut...
1    finally a transparant silicon case  thanks to ...
2    we love this  would you go talk makememories u...
3    im wired i know im george i was made that way ...
4    what amazing service  apple wont even talk to ...
Name: tweet, dtype: object

In [ ]:

!pip install nltk
import nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 310.6 kB/s eta 0:00:04
   ------------- -------------------------- 0.5/1.5 MB 310.6 kB/s eta 0:00:04
   ------------- -------------------------- 0.5/1.5 MB 310.6 kB/s eta 0:00:04
   ------------- -------------------------- 0.5/1.5 MB 310.6 kB/s eta 0:00:04
   -------------------- ------------------- 0.8/1.5 MB 299.6 kB/s e

In [38]:
import nltk

In [40]:
nltk.download('stopwords',download_dir='../static/model')

[nltk_data] Downloading package stopwords to ../static/model...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [43]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [47]:
data['tweet'] = data['tweet'].apply(lambda x: " ".join (x for x in x.split() if x not in sw))
data['tweet'].head()

0    fingerprint pregnancy test android apps beauti...
1    finally transparant silicon case thanks uncle ...
2    love would go talk makememories unplug relax i...
3    im wired know im george made way iphone cute d...
4    amazing service apple wont even talk question ...
Name: tweet, dtype: object

In [48]:
from nltk.stem import PorterStemmer 
ps = PorterStemmer()

In [51]:
data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split())) 
data['tweet'].head()

0    fingerprint pregnanc test android app beauti c...
1    final transpar silicon case thank uncl yay son...
2    love would go talk makememori unplug relax iph...
3    im wire know im georg made way iphon cute dave...
4    amaz servic appl wont even talk question unles...
Name: tweet, dtype: object

In [52]:
data['tweet']

0       fingerprint pregnanc test android app beauti c...
1       final transpar silicon case thank uncl yay son...
2       love would go talk makememori unplug relax iph...
3       im wire know im georg made way iphon cute dave...
4       amaz servic appl wont even talk question unles...
                              ...                        
7915    live loud lol liveoutloud selfi smile soni mus...
7916    would like wish amaz day make everi minut coun...
7917    help love year old neighbor ipad morn made rea...
7918    final got smart pocket wifi stay connect anyti...
7919    appl barcelona appl store bcn barcelona travel...
Name: tweet, Length: 7920, dtype: object

In [58]:
from collections import Counter
vocab = Counter()

In [61]:
for sentence in data['tweet']:
    vocab.update(sentence.split())

In [66]:
tokens = [key for key in vocab if vocab[key]>10]


In [70]:
def save_vocabulary(lines,filename):
    data = "\n".join(lines)
    file = open(filename,"w",encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(tokens,"../static/model/vacabulary.txt")

In [71]:
x=data['tweet']
y=data['label']

In [75]:

!pip install scikit-learn

   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.8 MB 345.3 kB/s eta 0:00:25
   -- ------------------------------------- 0.5/8.8 MB 345.3 kB/s eta 0:00:25
   -- ------------------------------------- 0.5/8.8 MB 345.3 kB/s eta 0:00:25
   --- ------------------------------------ 0.8/8.8 MB 311.7 kB/s eta 0:00:26
   --- ------------------------------------ 0.8/8.8 MB 311.7 kB/s eta 0:00:26
   --- ------------------------------------ 0.8/8.8 MB 311

In [78]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

3788    0
693     1
2638    1
6813    0
4311    0
       ..
1517    0
4847    0
3465    0
1189    0
6221    0
Name: label, Length: 1584, dtype: int64